In [1]:
from pandas import DataFrame, read_csv
import pandas as pd #this is how I usually import pandas
import matplotlib.pyplot as plt #for plot
import nltk
#read_csv?
Location = r'../twitter_data/train2017.tsv'
df = pd.read_csv(Location,delimiter = '\t',names=['id','id2','tag','text'])


In [2]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def my_concat(mylist):
    str = ""
    for word in mylist:
        str+= " "+word
    return str

from nltk.stem import  WordNetLemmatizer


# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()


# 3. Lemmatize a Sentence with the appropriate POS tag
list_tweets = []
for line in df.text:
    kati = my_concat([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(line)])
    list_tweets.append(kati)
    #print(kati)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer( max_features=250, stop_words='english') 
tfidf = tfidf_vectorizer.fit_transform(list_tweets)
print( tfidf.shape) #the output is a numpy array of features
iris_X = tfidf.toarray()

(27958, 250)


In [4]:
Location = r'../twitter_data/test2017.tsv'
df_res = pd.read_csv(Location,delimiter = '\t',names=['id','id2','tag','text'])
                 
res_tweets = []
for line_res in df_res.text:
    res_tmp = my_concat([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(line_res)])
    res_tweets.append(res_tmp)
                     
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer( max_features=250, stop_words='english') 
tfidf = tfidf_vectorizer.fit_transform(res_tweets)
print( tfidf.shape) #the output is a numpy array of features
                     
res_arr = tfidf.toarray()


(12284, 250)


In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
print(knn)

iris_Y = df.tag
knn.fit(iris_X , iris_Y)



KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [6]:
from sklearn import metrics

result = knn.predict(res_arr)
Location = r'../twitter_data/SemEval2017_task4_subtaskA_test_english_gold.txt'
df_cor = pd.read_csv(Location,delimiter = '\t',names=['id','tag'])
print(metrics.accuracy_score(result, df_cor.tag))

0.35346792575708236
